# pymongo基本操作

### 连接mongodb(无auth验证)

无验证式连接数据库通常是在第一次连接数据库时使用到,此时没有建立数据库,没有配置用户角色和读写权限.

In [26]:
import pymongo

# 仅仅创建mongodb连接,但未验证,没有连接到具体的数据库,相当于打开一个shell
client = pymongo.MongoClient(host='127.0.0.1', port=27017)
# 创建数据库
db = pymongo.database.Database(client, 'testdb')
db.command("updateUser", "arvin", pwd="arvin123")

# 创建管理员用户,密码和权限
db.command("createUser", "admin", pwd="admin@123456.", roles=["root"])
# 创建普通用户,密码和读取权限
db.command("createUser", "arvin", pwd="arvin@123456.", roles=["readWrite"])
# 创建普通用户,密码和只读权限
db.command("createUser", "xiaoming", pwd="xiaoming@123456.", roles=["read"])
# 创建普通用户,密码和只写权限
db.command("createUser", "xiaohong", pwd="xiaohong@123456.", roles=["write"])
# 修改用户密码
db.command("updateUser", "xiaoming", pwd="xiaoming@123.")
# 修改用户读取权限
db.command("updateUser", "xiaoming", roles=["readWrite"])

{'ok': 1.0}

### mongodb连接(带有auth验证)

In [ ]:
import pymongo

try:
    # Python 3.x
    from urllib.parse import quote_plus
except ImportError:
    # Python 2.x
    from urllib import quote_plus

MONGODB = {
    "user": "arvin",
    "passwd": "arvin@123456.",
    "host": "127.0.0.1:27017",
    "dbname": "testdb"
}

# 方式一
# 以类的方式创建mongodb的实例
class ConnectMongo(object):
    
        def __init__(self, MONGODB):
        self.client = MongoClient(
            'mongodb://{}:{}@{}/{}'.format(MONGODB['user'],
                                           MONGODB['passwd'],
                                           MONGODB['host'],
                                           MONGODB['dbname']),
            connect=False
        )
        self.mongodb = self.client[MONGODB['dbname']]

    def __del__(self):
        self.client.close()
        
# 方式二        
def connect_mongo(MONGODB):
    client = MongoClient(
        'mongodb://{}:{}@{}/{}'.format(MONGODB['user'],
                                       MONGODB['passwd'],
                                       MONGODB['host'],
                                       MONGODB['dbname']),
        connect=False)
    mongodb = client[MONGODB['dbname']]
    return mongodb

# 方式三: 处理特殊字符 
def connect_mongo(mongo_param):
    uri =  'mongodb://{}:{}@{}/{}'.format(quote_plus(mongo_param['user']),
                                          quote_plus(mongo_param['passwd']),
                                          mongo_param['host'],
                                          mongo_param['dbname'])
    try:
        client = pymongo.MongoClient(uri, connect=False)
        mongodb = client[mongo_param['dbname']]
        return mongodb
    except Exception as e:
        print(e)

**注意:** 
1. 方式二存在内存泄漏问题,因为创建的client没有在程序结束后close掉.
2. 当用户名或密码带有 ‘:’, ‘/’, ‘+’ 和 ‘@’ 这些字符时需要使用百分比编码,否则连接报错: `Username and password must be escaped according to RFC 3986, use urllib.parse.quote_plus().`

### 插入数据(insert)

In [32]:
import pymongo

MONGODB = {
    "user": "arvin",
    "passwd": "arvin123",
    "host": "127.0.0.1:27017",
    "dbname": "testdb"
}


def connect_mongo(mongo_param):
    uri =  'mongodb://{}:{}@{}/{}'.format(quote_plus(mongo_param['user']),
                                          quote_plus(mongo_param['passwd']),
                                          mongo_param['host'],
                                          mongo_param['dbname'])
    try:
        client = pymongo.MongoClient(uri, connect=False)
        mongodb = client[mongo_param['dbname']]
        return mongodb
    except Exception as e:
        print(e)
        
db = connect_mongo(MONGODB)

for i in range(10):
    
    doc = {'jan':[], 'feb': [], 'mar':[], 'apr':[], 'may':[], 'jun': [], 'jul':[], 'aug':[], 'sep':[], 'oct':[], 'nov':[], 'dec':[] }
    db['res_score'].insert_one(doc)

### 删除数据(delete)

In [33]:
import pymongo

MONGODB = {
    "user": "arvin",
    "passwd": "arvin123",
    "host": "127.0.0.1:27017",
    "dbname": "testdb"
}


def connect_mongo(mongo_param):
    uri =  'mongodb://{}:{}@{}/{}'.format(quote_plus(mongo_param['user']),
                                          quote_plus(mongo_param['passwd']),
                                          mongo_param['host'],
                                          mongo_param['dbname'])
    try:
        client = pymongo.MongoClient(uri, connect=False)
        mongodb = client[mongo_param['dbname']]
        return mongodb
    except Exception as e:
        print(e)
        
db = connect_mongo(MONGODB)

# 删除一条数据
db['res_score'].delete_one({})

# 删除全部数据
db['res_score'].delete_many({})


### 更新数据(update)

一般更新只更新最先找到的一条记录

#### 批量更新

批量更新只需要将第3个参数设为true即可

In [ ]:
db.getCollection('crawler_urls').update({'site': 'https://www.lljr.com'}, {$set: {"site" : "https://2code.top"}}, true)